# More Parallelism with Cython

Before jumping straight into more parallel programming with Cython, let's recap Python's _Global Interpreter Lock_ or GIL for short.

The GIL ensures that only one OS-level thread can execute Python bytecde at a time during the execution of a Python programming running with the CPython virtual machine (or 'interpreter', in Python parlance)

## Key points about the GIL

- The GIL affects not just the Python-level code the user writes, but also the Python/C API as a whole.

- The GIL is there for a reason - the memory management in CPython is not _thread-safe_ - it was not designed to handle deletion/creation of references to Python objects in a multithreaded world. 

- As a curious evolution of CPython, some other features in CPython now depend on the fact that the GIL exists, so removing is not a trivial task. 

- The GIL is specific to CPython. Other implementation of Python such as PyPy, Jython, and Iron-Python have no need for a GIL.

- C code that does not work with Python objects can be run without the GIL in effect. This is sometimes referred to as _releasing the GIL_ in interfaces that support it. **Parallel programming in Cython relies on this feature.**

Cython code is compiled ahead-of-time, not interpreted, so it is not running Python bytecode. Since we can use Cython to create C-only portions of code that are not tied to a Python object, we can release the global interpreter lock when working with the C-only parts of Cython. So we can use Cython to bypass the GIL and achieve true thread-based parallelism.

Cython provides two mechanisms for controlling the GIL: the `nogil` function attribute and the `with nogil:` context manager.

## The `nogil` function attribute

We can specifiy within Cython code that individual functions can be run with the GIL released. Any such function indicated as `nogil`-safe should be from an external library or declared as `cdef` or `cpdef` - i.e. a C-style function definition in Cython. A `def` function cannot be used with the `nogil` function attribute since these are by definition Python objects. (Funtions are first-class objects in Python).

The `nogil` function attribute is declared in the function's signature.

In [1]:
%load_ext cython

In [3]:
%%cython
cdef int kernel(double complex z, double complex z_max, int n_max) nogil:
    # ... do things
    pass

The `nogil` attribute is placed at the end of the function declaration, before the terminating colon. Inside the body of our function we must not create or interact with any Python objects. The Cython compiler does a good job of ensuring at compile time that the function will not accept or return Python objects, but it cannot guarantee this 100%, so the programmer should also be careful not to do this themselves.

For example, we could sneak in a Python object into a nogil function by casting it to a `void` pointer type.

### Declaring external functions `nogil`

Existing, external C++ and C functions can also be declared `nogil`. For example:


In [4]:
%%cython
cdef extern from "math.h":
    double sin(double x) nogil
    double cos(double x) nogil
    double tan(double x) nogil
    # ...etc

In many cases, external C or C++ libraries do not interact with Python objects at all. In that case, we can save ourselves some typing by declaring every function in an extern block as `nogil` by placing the `nogil` declaration in the `cdef extern from` line:

In [5]:
%%cython
cdef extern from "math.h" nogil:
    double sin(double x)
    double cos(double x)
    double tan(double x)
    # ...etc

When `nogil` is used in this way as a function attribute, it simply permits the function to be called without the GIL, it is still up to use to release the GIL before calling the nogil function. To do this, we use the `nogil` context manager.

## Using the `with nogil` context manager

To release the GIL, Cython must generate the appropriate Python/C API calls. Once the GIL has been released, it must be reacquired before interacting with Python objects. The Python `with` context manager is a natural fit for this.

In [8]:
%%cython

# Reminder of our funcion
cdef int kernel(double complex z, double complex z_max, int n_max) nogil:
    # ... do things
    pass

# Initialise the C-arguments...
cdef int result
cdef double z
cdef double z_max
cdef int n_max

# Nogil context manager
with nogil:
    result = kernel(z, z_max, n_max)

# GIL now reacquired through the with context manager.
print(result)  # Safe to call a Pyton function now.

0


The `with` statement acts as a context manager for the `nogil` statement, and will return the GIL once the with-block is exited. The argument types and return types for the for our `kernel` function are C data types by necessity, so we are not creating or interacting with any Python objects. If we try to interact with Python objects within a `with nogil1` block, Cython will issue a compile-time error. For example, we could not place a standard Python `print()` function inside a `nogil` block, as the argument is co-erced to a Python object.

One further use of the `with nogil` block is to release the GIL during blocking operations, i.e. CPU or I/O bound operations, allowing other Python threads to execute while another expensive operation executes concurrently.